In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:

!pip install segmentation-models-pytorch --no-cache-dir

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 15.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.3/121.3 kB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 192.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 311.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 280.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 369.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 193.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 290.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 222.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 263.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 177.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import os
import json
import cv2
import segmentation_models_pytorch as smp
from safetensors.torch import load_file
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.metrics import jaccard_score, precision_score, recall_score, f1_score



In [ ]:
config_path = "/content/drive/MyDrive/path_to_model/config.json"
with open(config_path, "r") as f:
    model_config = json.load(f)

# Initialize the model
model = smp.Linknet(
    encoder_name=model_config["encoder_name"],
    encoder_depth=model_config["encoder_depth"],
    encoder_weights=model_config["encoder_weights"],
    decoder_use_batchnorm=model_config["decoder_use_batchnorm"],
    in_channels=model_config["in_channels"],
    classes=model_config["classes"],
    activation=None  # Remove activation, since loss function will handle logits
).cuda()

# Load model weights
weights_path = "/content/drive/MyDrive/path_to_model/model.safetensors"
state_dict = load_file(weights_path)
model.load_state_dict(state_dict)
model.train()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

Linknet(
  (encoder): TimmUniversalEncoder(
    (model): FeatureListNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (drop_block): Identity()
          (act1): ReLU(inplace=True)
          (aa): Identity()
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act2): ReLU(inplace=True)
        )
        (1): BasicBlock(
     

In [ ]:
# Define Dataset
class ConjunctivaDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.image_filenames = sorted(os.listdir(image_dir))
        self.mask_filenames = sorted(os.listdir(mask_dir))
        self.transform = transform

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_filenames[idx])
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Load mask
        mask_path = os.path.join(self.mask_dir, self.mask_filenames[idx]) if idx < len(self.mask_filenames) else None
        if mask_path and os.path.exists(mask_path):
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            mask = mask / 255.0  # Normalize mask to [0,1]
        else:
            mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.float32)

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask.float().unsqueeze(0)  # Add channel dim

# Define Transformations
transform = A.Compose([
    A.Resize(256, 256),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
], is_check_shapes=False)

In [ ]:
dataset = ConjunctivaDataset(
    "/content/drive/MyDrive/dataset/eye_images",
    "/content/drive/MyDrive/dataset/cropped_masks",
    transform=transform
)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# Define Loss Function (BCEWithLogitsLoss)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Define Metrics
def dice_score(pred, target, threshold=0.5):
    pred = (pred > threshold).astype(np.uint8)
    target = (target > 0.5).astype(np.uint8)
    intersection = np.sum(pred * target)
    return (2. * intersection) / (np.sum(pred) + np.sum(target) + 1e-8)

def iou_score(pred, target, threshold=0.5):
    pred = (pred > threshold).astype(np.uint8)
    target = (target > 0.5).astype(np.uint8)
    return jaccard_score(target.flatten(), pred.flatten(), average="binary")

def precision(pred, target, threshold=0.5):
    pred = (pred > threshold).astype(np.uint8)
    target = (target > 0.5).astype(np.uint8)
    return precision_score(target.flatten(), pred.flatten(), zero_division=1)

def recall(pred, target, threshold=0.5):
    pred = (pred > threshold).astype(np.uint8)
    target = (target > 0.5).astype(np.uint8)
    return recall_score(target.flatten(), pred.flatten(), zero_division=1)

def f1(pred, target, threshold=0.5):
    pred = (pred > threshold).astype(np.uint8)
    target = (target > 0.5).astype(np.uint8)
    return f1_score(target.flatten(), pred.flatten(), zero_division=1)


In [ ]:
import torch
import numpy as np

# Define number of epochs
num_epochs = 25
patience = 5  # Early stopping patience
best_dice = 0  # Track best Dice Score
counter = 0  # Counter for early stopping

# Define optimizer and scheduler
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, factor=0.5)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for images, masks in dataloader:
        images, masks = images.cuda(), masks.cuda()

        optimizer.zero_grad()
        outputs = model(images)

        loss = criterion(outputs, masks)  # Compute loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)

    # Evaluation
    model.eval()
    total_dice, total_iou, total_acc, total_precision, total_recall, total_f1 = 0, 0, 0, 0, 0, 0
    num_samples = 0

    with torch.no_grad():
        for images, masks in dataloader:
            images, masks = images.cuda(), masks.cuda()
            outputs = model(images)
            outputs = torch.sigmoid(outputs).cpu().numpy()
            masks = masks.cpu().numpy()

            for i in range(len(outputs)):
                total_dice += dice_score(outputs[i], masks[i])
                total_iou += iou_score(outputs[i], masks[i])
                total_precision += precision(outputs[i], masks[i])
                total_recall += recall(outputs[i], masks[i])
                total_f1 += f1(outputs[i], masks[i])
                total_acc += np.mean((outputs[i] > 0.5) == (masks[i] > 0.5))  # Accuracy calculation
                num_samples += 1

    avg_dice = total_dice / num_samples
    avg_iou = total_iou / num_samples
    avg_precision = total_precision / num_samples
    avg_recall = total_recall / num_samples
    avg_f1 = total_f1 / num_samples
    avg_acc = total_acc / num_samples  # Average accuracy

    # Reduce LR if needed
    scheduler.step(avg_dice)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Training Loss: {avg_loss:.4f}")
    print(f"Model Evaluation:")
    print(f"Dice Score: {avg_dice:.4f}")
    print(f"IoU Score: {avg_iou:.4f}")
    print(f"Accuracy: {avg_acc:.4f}")
    print(f"Precision: {avg_precision:.4f}")
    print(f"Recall: {avg_recall:.4f}")
    print(f"F1 Score: {avg_f1:.4f}")
    print("-" * 50)

    # Early stopping logic
    if avg_dice > best_dice:
        best_dice = avg_dice
        counter = 0  # Reset counter if performance improves
    else:
        counter += 1  # Increase counter if no improvement
        if counter >= patience:
            print(f"Early stopping at epoch {epoch+1} due to no improvement in Dice Score.")
            break  # Stop training if no improvement for `patience` epochs

Epoch 1/25
Training Loss: 0.5225
Model Evaluation:
Dice Score: 0.8302
IoU Score: 0.7162
Accuracy: 0.7402
Precision: 0.7829
Recall: 0.9006
F1 Score: 0.8302
--------------------------------------------------
Epoch 2/25
Training Loss: 0.4697
Model Evaluation:
Dice Score: 0.8446
IoU Score: 0.7382
Accuracy: 0.7637
Precision: 0.7919
Recall: 0.9208
F1 Score: 0.8446
--------------------------------------------------
Epoch 3/25
Training Loss: 0.4356
Model Evaluation:
Dice Score: 0.8464
IoU Score: 0.7399
Accuracy: 0.7784
Precision: 0.8264
Recall: 0.8817
F1 Score: 0.8464
--------------------------------------------------
Epoch 4/25
Training Loss: 0.4129
Model Evaluation:
Dice Score: 0.8579
IoU Score: 0.7574
Accuracy: 0.7971
Precision: 0.8380
Recall: 0.8924
F1 Score: 0.8579
--------------------------------------------------
Epoch 5/25
Training Loss: 0.3992
Model Evaluation:
Dice Score: 0.8465
IoU Score: 0.7400
Accuracy: 0.7907
Precision: 0.8751
Recall: 0.8331
F1 Score: 0.8465
---------------------